In [7]:
from NPLearner import NPLearner
from NPLearner import default_feature_func

import nltk

from nltk.classify import SklearnClassifier
from nltk.classify import MaxentClassifier, \
                        ConditionalExponentialClassifier,\
                        DecisionTreeClassifier, \
                        NaiveBayesClassifier, \
                        WekaClassifier
import numpy as np
import pandas as pd

PTB = "treebank/"


In [8]:
# defining experiments

subjects = {"Decision Tree Classifier": DecisionTreeClassifier,
            "Maximum Entorpy Classifier": MaxentClassifier,
            "Naive Bayes Classifier": NaiveBayesClassifier,
            "Conditional Exponential Classifier": ConditionalExponentialClassifier}


In [9]:
# Max = 100 iterations
experiment = NPLearner(PTB, subjects, default_feature_func, verbose=True, max_iter=100)

experiment.fit()

experiment.predict()

metrics = experiment.evaluate()

---------- TRAINING ----------
Training Decision Tree Classifier...
Finished.

Training Maximum Entorpy Classifier...
  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.562
             2          -0.38053        0.836
             3          -0.30256        0.871
             4          -0.26721        0.889
             5          -0.24510        0.901
             6          -0.22916        0.908
             7          -0.21674        0.915
             8          -0.20661        0.920
             9          -0.19807        0.924
            10          -0.19072        0.929
            11          -0.18430        0.932
            12          -0.17860        0.935
            13          -0.17351        0.937
            14          -0.16891        0.939
            15          -0.16473        0.941
            16          -0.16091        0.943
            17       